In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests, html
from tqdm import tqdm
import re
import json
from datetime import datetime

In [2]:
def all_page_link(p_start_url: str, p_debug: bool = False) -> list:
    all_urls = []
    url = p_start_url
        
    if p_debug:
        print(f'DEBUG {datetime.now()} all_page_link Start URL: {url}')
        
    while(url != None):            #Loop around all the required webpages and terminates when last page arive!
        all_urls.append(url)
        soup = BeautifulSoup(requests.get(url).text,"html.parser")
        next_links = soup.find_all(class_='flat-button lister-page-next next-page')    #Extracts the next page link.
        if (len(next_links) == 0):         # If their is no next page, it returns 0.
            url = None
        
            if p_debug:
                print(f'DEBUG {datetime.now()} all_page_link No more pages')
                
        else:
            next_page = "https://www.imdb.com" + next_links[0].get('href')
                
            url = next_page
        
            if p_debug:
                print(f'DEBUG {datetime.now()} all_page_link Next page: {url}')
                
    return all_urls

In [3]:
def extract_imdb_data(p_content: str, p_debug: bool = False) -> list:
        
    #if p_debug:
    #    print(f'DEBUG {datetime.now()} extract_imdb_data Current content:\n{p_content}')
    
    soup = BeautifulSoup(p_content, 'html.parser')

    soup_data = soup.find("script", type="application/ld+json").text

    imdb_data = json.loads(soup_data)
    
    l_movie_name = html.unescape(imdb_data['name'])
        
    if p_debug:
        print(f'DEBUG {datetime.now()} extract_imdb_data Data extracted from content:\n{imdb_data}')

    soup_oscars = soup.findAll("a", attrs={"class":"ipc-metadata-list-item__label ipc-metadata-list-item__label--link"})
        
    num_of_oscars = 0
    for i in soup_oscars:
        if re.search('Won(.+?)Oscars',i.text):
            num_of_oscars = int(re.findall(r'\d+',i.text)[0])

            if p_debug:
                print(f'DEBUG {datetime.now()} extract_imdb_data Found Oscars: {num_of_oscars} in: {i.text}')
            
    try:
        release_date = imdb_data['datePublished']
    except KeyError as ke:
        if p_debug:
            print(f'DEBUG {datetime.now()} extract_imdb_data Publish date was not found for: {l_movie_name}')
        release_date = 'N/A'
    
    l_return =  [l_movie_name,
                 release_date,
                 imdb_data['aggregateRating']['ratingValue'],
                 imdb_data['aggregateRating']['ratingCount'],
                 num_of_oscars]
        
    if p_debug:
        print(f'DEBUG {datetime.now()} extract_imdb_data Extracted data: {l_return}')
    
    return l_return

In [4]:
def extract_imdb_top_250_data(p_debug: bool = False) -> pd.DataFrame:
    
    l_top_250_url = "https://www.imdb.com/list/ls068082370/"
        
    if p_debug:
        print(f'DEBUG {datetime.now()} extract_imdb_top_250_data Starting on URL: {l_top_250_url}')
        
    t = tqdm(all_page_link(p_start_url=l_top_250_url, p_debug=p_debug))
    
    links_set = set()
    for i in t:
        soup = BeautifulSoup(requests.get(i).text,"html.parser")
        links = [a['href'] for a in soup.select('a[href]')]
        current_link_set = set(list(filter(lambda link: 'title/tt' in link, links)))
        
        if p_debug:
            print(f'DEBUG {datetime.now()} extract_imdb_top_250_data Adding links:\n{current_link_set}')
        
        links_set = links_set.union(current_link_set)
        
    if p_debug:
        print(f'DEBUG {datetime.now()} extract_imdb_top_250_data Current set of links:\n{links}')
        
    imdb_top_250_data = []
    for link in links_set:
        l_current_link = f'https://www.imdb.com{link}'
        
        if p_debug:
            print(f'DEBUG {datetime.now()} extract_imdb_top_250_data Extracting data from URL:{l_current_link}')
            
        l_content = requests.get(l_current_link).content
        imdb_top_250_data.append(extract_imdb_data(p_content=l_content, p_debug=p_debug))
        
    index = ["name", "release_date", "rating", "votes", "oscars"]

    df = pd.DataFrame(imdb_top_250_data,columns=index)
        
    if p_debug:
        print(f'DEBUG {datetime.now()} extract_imdb_top_250_data Result dataframe:\n{df}')
    
    return df

In [5]:
df = extract_imdb_top_250_data()

df.head(5)

DEBUG 2022-10-22 16:57:56.055289 extract_imdb_top_250_data Starting on URL: https://www.imdb.com/list/ls068082370/
DEBUG 2022-10-22 16:57:56.055289 all_page_link Start URL: https://www.imdb.com/list/ls068082370/
DEBUG 2022-10-22 16:58:04.914782 all_page_link Next page: https://www.imdb.com/list/ls068082370/?page=2
DEBUG 2022-10-22 16:58:12.457383 all_page_link Next page: https://www.imdb.com/list/ls068082370/?page=3
DEBUG 2022-10-22 16:58:17.624819 all_page_link No more pages


 33%|████████████████████████████                                                        | 1/3 [00:08<00:16,  8.47s/it]

DEBUG 2022-10-22 16:58:26.117725 extract_imdb_top_250_data Adding links:
{'/title/tt0266697/', '/title/tt0120737/', '/title/tt0033467/', '/title/tt0078748/', '/title/tt0060196/', '/title/tt0064116/', '/title/tt0120903/', '/title/tt0172495/', '/title/tt0253474/', '/title/tt0076759/', '/title/tt0021749/', '/title/tt0107290/', '/title/tt0986264/', '/title/tt0110413/', '/title/tt0052357/', '/title/tt0073486/', '/title/tt1853728/', '/title/tt0120586/', '/title/tt0830515/', '/title/tt0434409/', '/title/tt0169547/', '/title/tt0211915/', '/title/tt0034583/', '/title/tt0137523/', '/title/tt0071562/', '/title/tt0032553/', '/title/tt0087469/', '/title/tt0133093/', '/title/tt0266543/', '/title/tt0468569/', '/title/tt0047478/', '/title/tt0111161/', '/title/tt0103064/', '/title/tt0095327/', '/title/tt1205489/', '/title/tt0476735/', '/title/tt0114369/', '/title/tt0058777/', '/title/tt0090605/', '/title/tt1049413/', '/title/tt0209144/', '/title/tt3315342/', '/title/tt0482571/', '/title/tt1675434/', '/

 67%|████████████████████████████████████████████████████████                            | 2/3 [00:15<00:07,  7.70s/it]

DEBUG 2022-10-22 16:58:33.280401 extract_imdb_top_250_data Adding links:
{'/title/tt0114709/', '/title/tt0105236/', '/title/tt0055630/', '/title/tt0055031/', '/title/tt0042876/', '/title/tt0045152/', '/title/tt0018455/', '/title/tt0044741/', '/title/tt0031381/', '/title/tt0089881/', '/title/tt0457430/', '/title/tt0120735/', '/title/tt2106476/', '/title/tt0053604/', '/title/tt1280558/', '/title/tt0119217/', '/title/tt0050986/', '/title/tt0242519/', '/title/tt0050976/', '/title/tt0066921/', '/title/tt0361748/', '/title/tt0017925/', '/title/tt0081398/', '/title/tt0047296/', '/title/tt0046268/', '/title/tt2096673/', '/title/tt0032976/', '/title/tt0012349/', '/title/tt0040897/', '/title/tt0019254/', '/title/tt0363163/', '/title/tt0040522/', '/title/tt0059578/', '/title/tt1255953/', '/title/tt0053291/', '/title/tt0119488/', '/title/tt0071315/', '/title/tt0061512/', '/title/tt0118715/', '/title/tt0050212/', '/title/tt0060107/', '/title/tt3170832/', '/title/tt4430212/', '/title/tt0093058/', '/

...

DEBUG 2022-10-22 17:07:03.148213 extract_imdb_data Data extracted from content:
{'@context': 'https://schema.org', '@type': 'Movie', 'url': '/title/tt0246578/', 'name': 'Donnie Darko', 'image': 'https://m.media-amazon.com/images/M/MV5BZjZlZDlkYTktMmU1My00ZDBiLWFlNjEtYTBhNjVhOTM4ZjJjXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_.jpg', 'description': 'After narrowly escaping a bizarre accident, a troubled teenager is plagued by visions of a man in a large rabbit suit who manipulates him to commit a series of crimes.', 'review': {'@type': 'Review', 'itemReviewed': {'@type': 'CreativeWork', 'url': '/title/tt0246578/'}, 'author': {'@type': 'Person', 'name': 'Patuquitos'}, 'dateCreated': '2005-02-08', 'inLanguage': 'English', 'name': 'Nothing more than tricks... but, god, how they work.', 'reviewBody': 'Sweets. They are not nourishing, but they&apos;re delicious.\n\nThis is how I feel about this film. If I make a conscious attempt at explaining why I like &quot;Donnie Darko&quot;, I can see no real meat 

DEBUG 2022-10-22 17:07:07.433255 extract_imdb_data Data extracted from content:
{'@context': 'https://schema.org', '@type': 'Movie', 'url': '/title/tt2024544/', 'name': '12 Years a Slave', 'alternateName': '12 év rabszolgaság', 'image': 'https://m.media-amazon.com/images/M/MV5BMjExMTEzODkyN15BMl5BanBnXkFtZTcwNTU4NTc4OQ@@._V1_.jpg', 'description': 'In the antebellum United States, Solomon Northup, a free black man from upstate New York, is abducted and sold into slavery.', 'review': {'@type': 'Review', 'itemReviewed': {'@type': 'CreativeWork', 'url': '/title/tt2024544/'}, 'author': {'@type': 'Person', 'name': 'tigerfish50'}, 'dateCreated': '2013-09-22', 'inLanguage': 'English', 'name': 'Abducted to a Southern hell', 'reviewBody': 'Considering the social and economic importance of slavery in America&apos;s history, the scarcity of serious films depicting the daily life of slaves in the Confederate States is significant - especially since the after-effects from this shameful episode still

DEBUG 2022-10-22 17:07:11.795935 extract_imdb_data Data extracted from content:
{'@context': 'https://schema.org', '@type': 'Movie', 'url': '/title/tt0081505/', 'name': 'The Shining', 'alternateName': 'Ragyogás', 'image': 'https://m.media-amazon.com/images/M/MV5BYjAxN2YxYTYtOGU0MC00ZTFjLTg5NmYtNTA0M2NiOGY5NmNlXkEyXkFqcGdeQXVyMzQyMDgzOTU@._V1_.jpg', 'description': 'A family heads to an isolated hotel for the winter where a sinister presence influences the father into violence, while his psychic son sees horrific forebodings from both past and future.', 'review': {'@type': 'Review', 'itemReviewed': {'@type': 'CreativeWork', 'url': '/title/tt0081505/'}, 'author': {'@type': 'Person', 'name': 'akdjreview'}, 'dateCreated': '2021-12-28', 'inLanguage': 'English', 'name': 'Not so scary incredible film', 'reviewBody': 'Horror films are not my cup of tea. But The Shining has gotten me to reconsider this preference. With an incredible performance by Jack Nicholson as Jack Torrance, I might even sa

DEBUG 2022-10-22 17:07:15.702550 extract_imdb_data Data extracted from content:
{'@context': 'https://schema.org', '@type': 'Movie', 'url': '/title/tt0405159/', 'name': 'Million Dollar Baby', 'alternateName': 'Millió dolláros bébi', 'image': 'https://m.media-amazon.com/images/M/MV5BMTkxNzA1NDQxOV5BMl5BanBnXkFtZTcwNTkyMTIzMw@@._V1_.jpg', 'description': 'A determined woman works with a hardened boxing trainer to become a professional.', 'review': {'@type': 'Review', 'itemReviewed': {'@type': 'CreativeWork', 'url': '/title/tt0405159/'}, 'author': {'@type': 'Person', 'name': 'namashi_1'}, 'dateCreated': '2010-08-21', 'inLanguage': 'English', 'name': 'Superb!', 'reviewBody': 'Clint Eastwood is a legendary film personality. I am a great fan of his. As an actor, I had my reservations with him, but as a storyteller, I have truly inhaled him. &apos;Million Dollar Baby&apos; is amongst his finest films, a superb film, that almost reaches a Landmark level!\n\nAbout a hardened trainer/manager work

,name,release_date,rating,votes,oscars
0,Kill Bill: Vol. 1,2003-10-16,8.2,1107119,0
1,Toy Story,1996-03-28,8.3,986567,0
2,Reservoir Dogs,1992-09-02,8.3,1010468,0
3,Yôjinbô,1961-04-25,8.2,123051,0
4,Judgment at Nuremberg,1966-01-27,8.3,77831,2


In [12]:
df.sort_values('rating', ascending=False).head(20)

,name,release_date,rating,votes,oscars
81,The Shawshank Redemption,1995-05-25,9.3,2652692,0
237,The Godfather,1982-03-25,9.2,1838803,3
67,The Godfather Part II,1983-04-21,9.0,1259937,6
79,The Dark Knight,2008-08-07,9.0,2625024,2
166,The Lord of the Rings: The Return of the King,2004-01-08,9.0,1828479,11
146,Schindler's List,1994-03-10,9.0,1344139,7
207,12 Angry Men,1960-01-28,9.0,783247,0
124,Pulp Fiction,1995-04-13,8.9,2030575,0
6,The Lord of the Rings: The Fellowship of the Ring,2002-01-10,8.8,1856164,4
127,The Lord of the Rings: The Two Towers,2003-01-09,8.8,1651073,2


In [14]:
max_votes = df.sort_values('rating', ascending=False).head(20).max(axis = 0)['votes']

In [15]:
def oscars_calculator(p_num_of_oscars: int) -> float:
    if p_num_of_oscars == 0:
        return 0
    elif p_num_of_oscars > 0 and p_num_of_oscars < 3:
        return 0.3
    elif p_num_of_oscars > 2 and p_num_of_oscars < 6:
        return 0.5
    elif p_num_of_oscars > 5 and p_num_of_oscars < 11:
        return 1
    else:
        return 1.5

In [40]:
df['adjusted_rating1'] = df['rating'] + ((max_votes - df['votes']) // 100000  * -0.1)

In [38]:
df['adjusted_rating2'] =  [oscars_calculator(x) for x in df['oscars']]

In [41]:
df['adjusted_rating'] = df['adjusted_rating1'] + df['adjusted_rating2']

In [44]:
df = df.drop("adjusted_rating1", axis='columns')
df = df.drop("adjusted_rating2", axis='columns')

In [46]:
df.sort_values('adjusted_rating', ascending=False).head(20)

,name,release_date,rating,votes,oscars,adjusted_rating
166,The Lord of the Rings: The Return of the King,2004-01-08,9.0,1828479,11,9.7
81,The Shawshank Redemption,1995-05-25,9.3,2652692,0,9.3
183,Forrest Gump,1994-12-08,8.8,2054771,6,9.3
79,The Dark Knight,2008-08-07,9.0,2625024,2,9.3
158,Inception,2010-07-22,8.8,2326471,4,9.0
237,The Godfather,1982-03-25,9.2,1838803,3,8.9
67,The Godfather Part II,1983-04-21,9.0,1259937,6,8.7
146,Schindler's List,1994-03-10,9.0,1344139,7,8.7
6,The Lord of the Rings: The Fellowship of the Ring,2002-01-10,8.8,1856164,4,8.6
76,The Matrix,1999-08-05,8.7,1896539,4,8.5
